Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
# !wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
!wget --no-check-certificate www.cse.iitd.ac.in/~kskeshav/test.data.txt

--2021-10-28 11:39:38--  http://www.cse.iitd.ac.in/~kskeshav/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~kskeshav/test.data.txt [following]
--2021-10-28 11:39:39--  https://www.cse.iitd.ac.in/~kskeshav/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138961 (136K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>] 135.70K   244KB/s    in 0.6s    

2021-10-28 11:39:40 (244 KB/s) - ‘test.data.txt’ saved [138961/138961]



In [ ]:
!gdown https://drive.google.com/uc?id=1xRfEBiybZwKKzIQOxEAGECgNeFr-mkXJ

Downloading...
From: https://drive.google.com/uc?id=1xRfEBiybZwKKzIQOxEAGECgNeFr-mkXJ
To: /content/2018EE10493_A_model.zip
100% 27.1M/27.1M [00:00<00:00, 127MB/s] 


In [ ]:
!unzip ./2018EE10493_A_model.zip

Archive:  ./2018EE10493_A_model.zip
 extracting: data.zip                
   creating: NLP_A3/
   creating: NLP_A3/data/
   creating: NLP_A3/data/train/
  inflating: NLP_A3/data/train/train.gold.txt  
  inflating: NLP_A3/data/train/train.data.txt  
   creating: NLP_A3/data/validation/
  inflating: NLP_A3/data/validation/validation.data.txt  
  inflating: NLP_A3/data/validation/validation.gold.txt  
   creating: results/
   creating: results/lstm/
   creating: results/lstm/mnli/
  inflating: results/lstm/mnli/best-lstm-mnli-params.pt  
  inflating: results/lstm/mnli/train.log  
  inflating: results/lstm/mnli/vocab  
  inflating: train_pos.json          
  inflating: valid_pos.json          


In [ ]:
test_data_path = '/content/test.data.txt'

In [ ]:
model_file_path = './results/lstm/mnli'

In [ ]:
import dill

In [ ]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace

In [ ]:
import zipfile
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
import pandas as pd
import numpy as np
import random

In [ ]:
with open(os.path.join(model_file_path, "vocab"), 'rb') as f:
  sent = dill.load(f)

In [ ]:
def Dataset(batch_size, sent):
  file_ = pd.read_csv(test_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  file_['index_sent2'] = file_['index_sent1']
  for i in file_.index:
    idx1 = file_.at[i, 'index_sent1']
    file_.at[i, 'index_sent1'] = idx1[0]
    idx2 = file_.at[i, 'index_sent2']
    file_.at[i, 'index_sent2'] = idx2[-1]
  for i in file_.index:
    idx1 = file_.at[i, 'POS']
    if idx1[0] == 'N':
      file_.at[i, 'POS'] = 1
    else:
      file_.at[i, 'POS'] = 0
  file_.to_json('test_pos.json', orient = 'records', lines = True)
  load_model_lemma = spacy.load('en', disable = ['parser','ner'])
  def tokenize_lemma(text):
    doc = load_model_lemma(text)
    return [token.lemma_ for token in doc]

  idx_sent = Field(sequential=False, use_vocab=False)
  sent = sent
  category = Field(sequential=False, use_vocab=False)
  fields = {'word': ('w', sent), "POS":("pos", category), "index_sent1": ('idx1', idx_sent), "index_sent2": ('idx2', idx_sent), "sent1":('s1', sent), "sent2":('s2',sent)}
  test_data = TabularDataset.splits(path="", train="test_pos.json", format="json", fields=fields)

  test_iterator = BucketIterator.splits(test_data, batch_size=batch_size, device="cuda", sort_key = lambda x: len(x.s1), sort_within_batch=False, shuffle=False)[0]
  return test_iterator

In [ ]:
test_iter = Dataset(128, sent)

In [ ]:
def adjust_dim(word_idx, hidden_size):
  word_idx = torch.unsqueeze(word_idx, 0)
  word_idx = torch.unsqueeze(word_idx, 2)
  word_idx = torch.repeat_interleave(word_idx, hidden_size, dim=2)
  return word_idx

In [ ]:
class Context_word_embedding_model(nn.Module):
  def __init__(self, input_size, embed_size, hidden_size, num_layers, bidirectional, num_direction, dropout, device):
    super(Context_word_embedding_model, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_direction = num_direction
    self.embedding = nn.Embedding(input_size, embed_size,padding_idx= 1)
    self.dimension = hidden_size*num_direction
    self.device = device
    self.drop = nn.Dropout(p =0.1)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional = bidirectional, dropout = dropout)
    self.fc = nn.Linear(self.dimension, self.dimension)
    self.fc_pos = nn.Linear(self.dimension, 1)
    self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    self.sig = nn.Sigmoid()

  def forward(self, batch):
    x = batch.s1.to(device=self.device)
    s2 = batch.s2.to(device=self.device)
    word_idx = batch.idx1.to(device=self.device)
    word_idx2 = batch.idx2.to(device=self.device)
    h0 = torch.zeros(self.num_layers*self.num_direction, x.size(1), self.hidden_size).to(self.device)
    c0 = torch.zeros(self.num_layers*self.num_direction, x.size(1), self.hidden_size).to(self.device)
    h02 = torch.zeros(self.num_layers*self.num_direction, s2.size(1), self.hidden_size).to(self.device)
    c02 = torch.zeros(self.num_layers*self.num_direction, s2.size(1), self.hidden_size).to(self.device)

    embedded_ = self.embedding(x)
    embedded2_ = self.embedding(s2)
    embedded = self.drop(embedded_)
    embedded2 = self.drop(embedded2_)
    outputs, _ = self.lstm(embedded, (h0,c0))
    outputs2, __ = self.lstm(embedded2, (h02,c02))
    word_idx = adjust_dim(word_idx, self.hidden_size*self.num_direction)
    word_idx2 = adjust_dim(word_idx2, self.hidden_size*self.num_direction)
    embed = torch.gather(outputs, 0, word_idx).squeeze()
    embed2 = torch.gather(outputs2, 0, word_idx2).squeeze()
    context_embedding = self.fc(embed)
    context_embedding2 = self.fc(embed2)
    out_sim = self.cos(context_embedding, context_embedding2)
    out_final = self.sig(out_sim)
    return out_final


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
input_size = len(sent.vocab)
embedding_size = 300
hidden_size = 200
num_layers = 1
bidirectional = False
num_direction = 1
dropout = 0
model = Context_word_embedding_model(input_size, embedding_size, hidden_size, num_layers, bidirectional, num_direction, dropout, device).to(device)

In [ ]:
checkpoint = torch.load('./results/lstm/mnli/best-lstm-mnli-params.pt')

In [ ]:
checkpoint['accuracy']

tensor(60.0313, device='cuda:0')

In [ ]:
model.load_state_dict(checkpoint['model_dict'])
model.eval()

Context_word_embedding_model(
  (embedding): Embedding(5739, 300, padding_idx=1)
  (drop): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(300, 200)
  (fc): Linear(in_features=200, out_features=200, bias=True)
  (fc_pos): Linear(in_features=200, out_features=1, bias=True)
  (cos): CosineSimilarity()
  (sig): Sigmoid()
)

In [ ]:
def test_model(test_iter, model):
  #model.eval()
  test_iter.init_epoch()
  y_pred = torch.zeros(1)
  with torch.no_grad():
    for batch_idx, batch in enumerate(test_iter):
      answer = model(batch)
      x = answer > 0.5
      predictions = list(map(int,x))
      predictions = torch.tensor(predictions).to(device)
      if batch_idx == 0:
        y_pred = predictions
      else:
        y_pred = torch.cat((y_pred,predictions))
  y_pred = y_pred.to('cpu')
  with open('output.txt', "w") as output_file:
    for y in y_pred:
      if y == 1:
        Y_write = 'T'
      elif y == 0:
        Y_write = 'F'
      output_file.write("%s\n" % Y_write)
    output_file.close()

In [ ]:
test_model(test_iter, model)

In [ ]:
## YOUR testing code

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-28 11:41:40--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-28 11:41:40--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-28 11:41:40 (64.7 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.48746081504702193


In [ ]:
!zip 2018EE10493_A.zip output.txt

  adding: output.txt (deflated 90%)
